In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

ModuleNotFoundError: No module named 'xgboost'

In [2]:
train_data = pd.read_csv("./smile_description_train.csv")
test_data = pd.read_csv("./smile_description_test.csv")

In [3]:
# Replacing Null values of train and test data with 0 
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

In [4]:
# Split into X and y
X = train_data.drop("label",axis=1)
y = train_data["label"]

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [6]:
def create_submission(predict,filename):
    sub_file = pd.read_csv("./data/sample_submission.csv")
    sub_file["Predicted"] = predict
    sub_file.to_csv(filename,index=False)
    print(filename," Created")

In [7]:
xgb_c = xgb.XGBClassifier(random_state=42,n_estimators=600)

In [16]:
def f1_score_xg_boost(X,y,learning_rate=None):
    skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
    cv =  skf.get_n_splits(X, y)
    scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
    cv_results = cross_validate(xgb.XGBClassifier(random_state=42,n_estimators=700,eta=learning_rate), X, y, cv=cv,scoring=scoring,verbose=3,n_jobs=-1)
    print("F1 score with ",sum(cv_results["test_f1_score"])/5)
    print("Accuracy score with ",sum(cv_results["test_accuracy"])/5)

In [17]:
import numpy as np
def get_count_of_ones_and_twos(predict):
    print("Number of predicted ones",np.count_nonzero(predict==1))
    print("Number of predicted twos",np.count_nonzero(predict==2))

In [18]:
f1_score_xg_boost(X,y,0.35)
"""
n_estimators = 700
learning_rate = 0.35
F1 score with  0.9483140858093458
Accuracy score with  0.9100460248164841
n_estimators = 900
learning_rate = 0.4
F1 score with  0.9472493912457081
Accuracy score with  0.9082816785922123
"""

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 30.4min remaining: 45.6min


F1 score with  0.9483140858093458
Accuracy score with  0.9100460248164841


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 30.4min finished


'\nn_estimators = 700\nlearning_rate = 0.35\nF1 score with  0.9483140858093458\nAccuracy score with  0.9100460248164841\nn_estimators = 900\nlearning_rate = 0.4\nF1 score with  0.9472493912457081\nAccuracy score with  0.9082816785922123\n'

In [19]:
xgb_c = xgb.XGBClassifier(random_state=42,n_estimators=700,eta=0.35)

In [20]:
xgb_c.fit(X,y)
predict = xgb_c.predict(test_data)
predict_real = label_encoder.inverse_transform(predict)

[CV] END  accuracy: (test=0.911) f1_score: (test=0.949) precision: (test=0.938) recall: (test=0.960) total time=30.3min
[CV] END  accuracy: (test=0.913) f1_score: (test=0.950) precision: (test=0.936) recall: (test=0.964) total time=30.4min
[CV] END  accuracy: (test=0.908) f1_score: (test=0.947) precision: (test=0.935) recall: (test=0.960) total time=30.4min
[CV] END  accuracy: (test=0.912) f1_score: (test=0.949) precision: (test=0.937) recall: (test=0.962) total time=30.4min
[CV] END  accuracy: (test=0.907) f1_score: (test=0.946) precision: (test=0.934) recall: (test=0.959) total time=30.4min


In [21]:
get_count_of_ones_and_twos(predict_real)

Number of predicted ones 1361
Number of predicted twos 9633


In [22]:
create_submission(predict_real,"submission_1_mar_5.csv")

submission_1_mar_5.csv  Created
